In [1]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import matplotlib.pyplot as plt
import os
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import acquire
import prepare

In [3]:
titan_df = acquire.get_titanic_data()
titan_df = titan_df.drop(titan_df.columns[0], axis = 1)
titan_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [9]:
titan_df = prepare.clean_titanic(titan_df)

In [11]:
titan_df = titan_df.drop(columns = ['sex','embark_town'])

In [12]:
train, val, test = prepare.train_val_test(titan_df, 'survived') #splits up the data
train.head()

,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
748,0,1,1,0,53.1000,0,1,0,1
45,0,3,0,0,8.0500,1,1,0,1
28,1,3,0,0,7.8792,1,0,1,0
633,0,1,0,0,0.0000,1,1,0,1
403,0,3,1,0,15.8500,0,1,0,1


In [13]:
x_train = train.drop(columns = ['survived'])
y_train = train['survived']
x_val = val.drop(columns = ['survived'])
y_val = val['survived']
x_test = test.drop(columns = ['survived'])
y_test = test['survived']

Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [7]:
seed = 42

rf_titan = RandomForestClassifier(max_depth = 10, random_state = seed, min_samples_leaf = 1, max_samples = 0.5)

In [20]:
rf_titan.fit(x_train, y_train)
for1 = rf_titan.fit(x_train, y_train)

Evaluate your results using the model score, confusion matrix, and classification report.

In [22]:
importance_df = pd.DataFrame({'feature': x_train.columns.tolist(), 
              'importance': rf_titan.feature_importances_})

In [23]:
importance_df.sort_values('importance', ascending = False)

,feature,importance
3,fare,0.396000
5,sex_male,0.308200
0,pclass,0.100537
2,parch,0.064735
1,sibsp,0.060262
7,embark_town_Southampton,0.032563
4,alone,0.024954
6,embark_town_Queenstown,0.012749


In [19]:
print(rf_titan.score(x_train, y_train),rf_titan.score(x_val, y_val))

0.9004815409309791 0.8134328358208955


In [24]:
train_preds = rf_titan.predict(x_train)

In [58]:
pd.DataFrame(classification_report(y_train, train_preds, output_dict = True))

,0,1,accuracy,macro avg,weighted avg
precision,0.894608,0.911628,0.900482,0.903118,0.901137
recall,0.950521,0.820084,0.900482,0.885302,0.900482
f1-score,0.921717,0.863436,0.900482,0.892577,0.899359
support,384.000000,239.000000,0.900482,623.000000,623.000000


In [ ]:
y_preds = for1.predict(x_train)

In [60]:
cm = confusion_matrix(y_train, y_preds)
cm

array([[365,  19],
       [ 43, 196]])

Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [26]:
y_preds = for1.predict(x_train)

TN, FP, FN ,TP = confusion_matrix(y_train, y_preds).ravel()

print (TN, FP, FN, TP)

ALL = TN + FP + FN + TP 

365 19 43 196


Run through steps increasing your min_samples_leaf and decreasing your max_depth.

In [73]:
metrics = []


def depth_n_leaf():
    msl = 1
    i = 3
    while msl <= 10:
        rf = RandomForestClassifier(max_depth = i,
                                    random_state = seed, min_samples_leaf = msl, max_samples = 0.5)
        rf.fit(x_train, y_train)
        train_acc = rf.score(x_train, y_train)
        val_acc = rf.score(x_val, y_val)
        output = {'max_depth': i, 'min_samples_leaf': msl, 'train_accuracy': train_acc, 
                  'validate_accuracy': val_acc}
        metrics.append(output)
        i = i +1
        if i == 11:
            msl = msl + 1
            i = 3


        
depth_n_leaf() 

metrics = pd.DataFrame(metrics)
metrics['difference'] = (metrics['train_accuracy'] - metrics['validate_accuracy'])    
metrics

,max_depth,min_samples_leaf,train_accuracy,validate_accuracy,difference
0,3,1,0.818620,0.828358,-0.009739
1,4,1,0.828250,0.835821,-0.007570
2,5,1,0.842697,0.820896,0.021801
3,6,1,0.855538,0.828358,0.027180
4,7,1,0.873194,0.820896,0.052299
5,8,1,0.892456,0.805970,0.086486
6,9,1,0.898876,0.805970,0.092906
7,10,1,0.900482,0.813433,0.087049
8,3,2,0.817014,0.828358,-0.011344
9,4,2,0.831461,0.835821,-0.004360


What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

After making a few models, which one has the best performance (or closest metrics) on both train and validate?